In [26]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [27]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        # create a container of dictionaries.
        self.logs = defaultdict(list)
    
    # log the message for a particular experiment in its 
    # respective dictionary
    def log(self, index, message):
        self.logs[index].append(message)
    
    # fetch all logs as collection
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(9):
        # pretend this is an experiment that produces a nine results for 
        # experiment result; in our case it's just a simple message 
        time.sleep(1)
        # Push a logging message to the actor.
        exp_key = f"expermint-{experiment_index}"
        logging_actor.log.remote(exp_key, 'On iteration {}'.format(i))  

In [28]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='2.0.0rc0', ray_commit='a0588094ec52b45a878f59e98258cd5e90f4ec36', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-04_13-42-56_336257_8725/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-04_13-42-56_336257_8725/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-08-04_13-42-56_336257_8725', 'metrics_export_port': 56318, 'gcs_address': '127.0.0.1:57803', 'address': '127.0.0.1:57803', 'dashboard_agent_listen_port': 52365, 'node_id': '84bd6d9520ae20acf0b737c7baccf043f4cc63a1887663e3c4bb08e1'})

Run three experiments.

In [29]:
logging_actor = LoggingActor.remote()
experiment_ids = []
# Run three different experiments
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

### Fetch the results 

For each experement, we will have 9 iteration results

In [32]:
for i in range(3):
    time.sleep(1)
    logs = logging_actor.get_logs.remote()
    print(ray.get(logs))

{'expermint-0': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'expermint-2': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'expermint-1': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8']}
{'expermint-0': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'expermint-2': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'expermint-1': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4